In [1]:
# Importing libraries
import pandas as pd
import numpy as np
import os

In [2]:
# Loading data
basics_path='title.basics (1).tsv.gz'
akas_path='title.akas.tsv.gz'
ratings_path='title.ratings.tsv.gz'
basics=pd.read_csv(basics_path,sep='\t',low_memory=False)
akas=pd.read_csv(akas_path,sep='\t',low_memory=False)
ratings=pd.read_csv(ratings_path,sep='\t',low_memory=False)

In [3]:
# Previewing data
display(basics.head(),akas.head(),ratings.head())
print(basics.info(),akas.info(),ratings.info())

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


,tconst,averageRating,numVotes
0,tt0000001,5.7,1966
1,tt0000002,5.8,263
2,tt0000003,6.5,1808
3,tt0000004,5.6,178
4,tt0000005,6.2,2607


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9808970 entries, 0 to 9808969
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 673.5+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35736618 entries, 0 to 35736617
Data columns (total 8 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   titleId          object
 1   ordering         int64 
 2   title            object
 3   region           object
 4   language         object
 5   types            object
 6   attributes       object
 7   isOriginalTitle  object
dtypes: int64(1), object(7)
memory usage: 2.1+ GB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1306416 entries, 0 to 1306415
Data

# Preprocessing

## BASICS

In [4]:
# Replace "\N" with np.nan
basics.replace({'\\N':np.nan},inplace=True)

In [5]:
# Eliminate movies that are null for runtimeMinutes
basics = basics[basics['runtimeMinutes'].notnull()]
basics.runtimeMinutes.isnull().sum()

0

In [6]:
# Eliminate movies that are null for genre
basics = basics[basics['genres'].notnull()]
basics.genres.isnull().sum()

0

In [7]:
# keep only titleType==Movie
basics=basics[basics['titleType']=='movie']
basics.titleType.unique()

array(['movie'], dtype=object)

In [8]:
# keep startYear 2000-2022
basics['startYear']=pd.to_numeric(basics['startYear'], errors='coerce')
# basics=basics[basics['startYear'].astype('int64')]
basics=basics[basics['startYear'] > 1999]
basics=basics[basics['startYear'] < 2023]
basics.startYear.unique()

array([2021., 2001., 2020., 2018., 2022., 2005., 2002., 2009., 2017.,
       2000., 2006., 2004., 2008., 2007., 2003., 2012., 2010., 2013.,
       2011., 2015., 2016., 2014., 2019.])

In [9]:
# Eliminate movies that include "Documentary" in genre 
# Exclude movies that are included in the documentary category.
is_documentary = basics['genres'].str.contains('documentary',case=False)
basics = basics[~is_documentary]
basics.genres.unique() # making sure no values contain 'documentary'

array(['Comedy,Fantasy,Romance', 'Drama', 'Drama,War',
       'Comedy,Horror,Sci-Fi', 'Comedy,Drama,Fantasy', 'Drama,Romance',
       'Biography,Drama', 'Comedy,Mystery', 'Drama,Fantasy', 'Comedy',
       'Adventure', 'Musical,Romance', 'Action,Adventure,Drama', 'Action',
       'Crime,Thriller', 'Comedy,Fantasy', 'Action,Crime,Drama',
       'Action,Thriller', 'Comedy,Drama,Romance', 'Drama,Music,Romance',
       'Comedy,Horror,Mystery', 'Crime,Drama,Thriller', 'Comedy,Drama',
       'Action,Adventure,Animation', 'Comedy,Romance', 'Action,Comedy',
       'Drama,Thriller', 'Comedy,Drama,Sci-Fi',
       'Adventure,Family,Fantasy', 'Drama,History', 'Drama,History,War',
       'Adventure,Animation,Comedy', 'Action,Adventure,Fantasy',
       'Action,Drama,Sci-Fi', 'Biography,Drama,Romance',
       'Horror,Mystery,Thriller', 'Comedy,Drama,Thriller',
       'Animation,Family,Musical', 'Drama,Mystery,Thriller',
       'Action,Adventure,Thriller', 'Action,Horror,Sci-Fi',
       'Action,Adventu

In [10]:
# filtering out non-US movies in AKAs table before filtering Basics table
akas=akas[akas['region']=='US']

In [11]:
# Keeping only US movies using filtered AKAs table
# Filter the basics table down to only include the US by using the filter akas dataframe
keepers =basics['tconst'].isin(akas['titleId'])
basics = basics[keepers]
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34803,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
61116,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
67669,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
86801,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
93938,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama
...,...,...,...,...,...,...,...,...,...
9808108,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019.0,NaN,74,Drama
9808503,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,2019.0,NaN,97,"Comedy,Drama,Fantasy"
9808643,tt9916170,movie,The Rehearsal,O Ensaio,0,2019.0,NaN,51,Drama
9808652,tt9916190,movie,Safeguard,Safeguard,0,2020.0,NaN,95,"Action,Adventure,Thriller"


## AKAS

In [12]:
# Replace "\N" with np.nan
akas.replace({'\\N':np.nan},inplace=True)

In [13]:
# ## NOT SURE IF THIS IS NEEDED AS THE INSTRUCTIONS ARE UNCLEAR; LEAVING CODE COMMENTED

# # Filtering out movie ids not shared in other dataframes:
# keep=akas['titleId'].isin(basics['tconst'])
# akas=akas[keep]
# akas

## Ratings

In [14]:
# Replace "\N" with np.nan (if any)
if ~ratings.isin(['\\N']).any().any():
    print("No '\\N' values.")
else:
    ratings.replace({'\\N':np.nan},inplace=True)

No '\N' values.


In [15]:
# Keeping only US movies using filtered AKAs table
keep=ratings['tconst'].isin(akas['titleId'])
ratings=ratings[keep]
ratings

,tconst,averageRating,numVotes
0,tt0000001,5.7,1966
1,tt0000002,5.8,263
4,tt0000005,6.2,2607
5,tt0000006,5.2,181
6,tt0000007,5.4,816
...,...,...,...
1306377,tt9916200,8.1,229
1306378,tt9916204,8.1,262
1306385,tt9916348,8.1,18
1306386,tt9916362,6.4,5307


# Creating Data Folder

In [16]:
os.makedirs('Data/',exist_ok=True)
# confirming 
os.listdir('Data/')

[]

In [17]:
# Previewing cleaned dataframes
display(basics.head(),akas.head(),ratings.head())

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34803,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
61116,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
67669,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
86801,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
93938,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama


,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0
14,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0
33,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0
36,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0
41,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0


,tconst,averageRating,numVotes
0,tt0000001,5.7,1966
1,tt0000002,5.8,263
4,tt0000005,6.2,2607
5,tt0000006,5.2,181
6,tt0000007,5.4,816


# Saving Compressed .csv.gz Files

In [18]:
## Save dataframe to Data/ folder as gzipped csvs.
basics.to_csv('Data/title_basics.csv.gz',compression='gzip',index=False) # added index=False to prevent 'Unnamed: 0' column
akas.to_csv('Data/title_akas.csv.gz',compression='gzip',index=False)
ratings.to_csv('Data/title_ratings.csv.gz',compression='gzip',index=False)

In [19]:
# Open saved file and preview again
basics = pd.read_csv('Data/title_basics.csv.gz', low_memory = False)
akas = pd.read_csv('Data/title_akas.csv.gz', low_memory = False)
ratings = pd.read_csv('Data/title_ratings.csv.gz', low_memory = False)
display(basics.head(),akas.head(),ratings.head())

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama


,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0.0
1,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0.0
2,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0.0
3,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0.0
4,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0.0


,tconst,averageRating,numVotes
0,tt0000001,5.7,1966
1,tt0000002,5.8,263
2,tt0000005,6.2,2607
3,tt0000006,5.2,181
4,tt0000007,5.4,816


In [20]:
# summary of dataframes
display(basics.info(),akas.info(),ratings.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86561 entries, 0 to 86560
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          86561 non-null  object 
 1   titleType       86561 non-null  object 
 2   primaryTitle    86561 non-null  object 
 3   originalTitle   86561 non-null  object 
 4   isAdult         86561 non-null  int64  
 5   startYear       86561 non-null  float64
 6   endYear         0 non-null      float64
 7   runtimeMinutes  86561 non-null  int64  
 8   genres          86561 non-null  object 
dtypes: float64(2), int64(2), object(5)
memory usage: 5.9+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1432914 entries, 0 to 1432913
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   titleId          1432914 non-null  object 
 1   ordering         1432914 non-null  int64  
 2   title            1432914 n

None

None

None